# Dynamic Investment Blueprint

### Goal: Call a stock market API to target companies of interest while instantly eradicating any companies that do not pass validation boundaries from our initial investment strategy.

### Why? Finding new talent to invest in may be tough and time consuming. By validating companies with real-time updated data provided by a third-party company's API, we can save time and focus fundamental investment strategies. Note: achievement of growth or value investing

### This Project: The project that you will read about here is only an illustration of what we can accomplish with my program. This program may be re-used with any technical parameters of interest and will output an analysis that can be used for projections and straight forward visualization diagrams. In this example we will call the API and validate any companies that fit our 'growth' investing scope (boundaries). Growth investing methods look for smaller companies that have potential for high growth in the future. Although this method is prone to possible loss, if we win there is a much better payout. So lets hundred-fold our investment!

### Part 1 - Picking a Strategy

#### Importing libraries and packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

#### Declare Functions here

##### Function to import a javascript formatted URL into variable

In [2]:
# This code was copied from,
# Credit: https://financialmodelingprep.com/developer/docs/companies-key-stats-free-api/#Python
#!/usr/bin/env python

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

##### Function to convert javascript into a pandas dataframe

In [3]:
def json_to_empty_dataframe(dictionary):
    columns = ['Company']
    # Create a list of the dictionary keys
    for keys in dictionary:
        columns.append(keys)
    
    # Create a dataframe with the columns
    initial_dataframe = pd.DataFrame(columns=columns)
    
    return initial_dataframe
    
#def json_fill_dataframe

##### Functions for Indicator Calculations

In [4]:
# Use this function to calculate eps ratio
#def earnings_per_share_ratio(total_profit, number_of_shares):

# Use this function to find the growth of the eps
#def earnings_per_share_growth(eps):

##### Other useful functions

In [124]:
# Function which extracts a big list of companies
def company_list(stock_list):
    # stock_list contains 'symbol', 'name', and 'price'
    final_list = []
    counter = 0
    # Loop through all stock name and append them
    for i in stock_list:
        for key in stock_list[counter]:
            #if key == 'price':
                if (stock_list[counter])['price'] < 0.5:
                    final_list.append((stock_list[counter])['symbol'])
        counter = counter + 1
        
    return final_list

# Function which adds values of the income statement to a stock
def income_statement_dataframe(income_statement):
    # income_statement contains multiple dictionaries with 'financials': 'Company', 'date', 'Revenue Growth', 'EPS'
    final_list = []
    temp = []
    counter = 0
    # Loop through all stock name and append them
    for i in income_statement:
        for key in income_statement[counter]:
            if (key == 'date') or (key == 'Revenue Growth') or (key == 'EPS'):
                independent = income_statement[counter]
                final_list.append(independent[key])
        temp.append(final_list)
        final_list = []
        counter = counter + 1
    clean_dataframe_income_statement = pd.DataFrame(temp, columns=['date', 'Revenue Growth', 'EPS'])
        
    return clean_dataframe_income_statement

# Function to match stock price with the dates in the main_dataframe then add the stock price column after match identification
def add_to_dataframe(stock_price, main_dataframe):

    counter = 0
    temp = []
    test = []
    prices = []

    for i in stock_price:
        temp.append((stock_price[counter])['date'])
        temp.append((stock_price[counter])['close'])
        test.append(temp)
        temp = []
        counter = counter + 1


    main_list = (main_dataframe['date'])

    # My Testing Strategy: (main_list[counter])[0:9] == (test[counter][0])[0:9]
    # If we have a hit, input test[counter][1] into a new list, then break loop and increment counter
    for dates in main_list:
        for loop_through in range(0, len(test)-1):
            if dates[0:9] == (test[loop_through][0])[0:9]:
                prices.append(test[loop_through][1])
                break

    # If information is missing from the API, fill the rest of the column in with 'NA'
    for i in range(0, 41-len(prices)):
        prices.append('NA')

    main_dataframe['Stock Price'] = prices
    return main_dataframe

add_to_dataframe(stock_data_stock_price, main_dataframe)


# Functions to input the stock symbol into the url so we can call the API
def name_API_company_profile(symbol, url):
    return url[:54] + symbol + url[54:]

def name_API_income_statement(symbol, url):
    return url[:69] + symbol + url[69:]

def name_API_stock_price(symbol, url):
    return url[:63] + symbol + url[63:]
##########################################################################

##### Call API from resource: financialmodelingprep.com

##### Get a ticker list of all companies in the stock market

In [6]:
# Import companies from API
url_stocks = ("https://financialmodelingprep.com/api/v3/company/stock/list")
stock_list = get_jsonparsed_data(url_stocks)

hierarchy = stock_list["symbolsList"]
# Convert json data into a list of all stocks
all_stocks = company_list(hierarchy)

#### Create an empty dataframe with relevant columns of multiple stocks

In [7]:
# Identify our data identifiers
ticker = "AAPL"

# Get API url for company profile
url_company_profile = ("https://financialmodelingprep.com/api/company/profile/?datatype=json") #54
# Add ticker symbol into data URL - this extracts data from the website reference
api_call = name_API_company_profile(ticker, url_company_profile)

# Take the website data from json into a pandas dataframe
js = get_jsonparsed_data(api_call)
stock_data = js[ticker]

test = json_to_empty_dataframe(stock_data)
test

,Company,Price,Beta,VolAvg,MktCap,LastDiv,Range,Changes,ChangesPerc,companyName,exchange,industry,website,description,CEO,sector,date_is_filed,date_bs_filed,date_cs_filed,image


##### Create a dataframe of one stock we want to look deeper into, this is where all the analysis of single stocks will come from

In [42]:
# Get API url for income statement information
url_income_statement = ("https://financialmodelingprep.com/api/v3/financials/income-statement/?period=quarter") #69

# Add ticker symbol into data URL - this extracts data from the website reference
api_call = name_API_income_statement(ticker, url_income_statement)

# Take the website data from json into a pandas dataframe
income_statement = get_jsonparsed_data(api_call)
stock_data_income_statement = income_statement["financials"]

# Create empty pandas dataframe
dataframe_income_statement = json_to_empty_dataframe(stock_data_income_statement[0])

# Delete All unecessary columns from pandas dataframe
clean_dataframe_income_statement = dataframe_income_statement.drop(columns=[ 'Company', 'Revenue', 'Cost of Revenue', 'Gross Profit', 'R&D Expenses', 'SG&A Expense',
                                                                            'Operating Expenses', 'Operating Income', 'Interest Expense', 'Earnings before Tax', 
                                                                            'Income Tax Expense', 'Net Income - Non-Controlling int', 'Net Income - Discontinued ops', 
                                                                            'Net Income', 'Preferred Dividends', 'Net Income Com', 'EPS Diluted', 'Weighted Average Shs Out', 'Weighted Average Shs Out (Dil)',
                                                                            'Dividend per Share', 'Gross Margin', 'EBITDA Margin', 'EBIT Margin', 'Profit Margin', 'Free Cash Flow margin',
                                                                            'EBITDA', 'EBIT', 'Consolidated Income', 'Earnings Before Tax Margin', 'Net Profit Margin'])

main_dataframe = income_statement_dataframe(stock_data_income_statement)

# Output Clean DataFrame
main_dataframe

,date,Revenue Growth,EPS
0,2019-06-29,0.0102,2.2
1,2019-03-30,-0.0511,2.47
2,2018-12-29,-0.0451,4.22
3,2018-09-29,0.1963,2.94
4,2018-06-30,0.173,2.36
5,2018-03-31,0.1558,2.75
6,2017-12-30,0.1269,3.92
7,2017-09-30,0.1222,2.09
8,2017-07-01,0.072,1.68
9,2017-04-01,0.0463,2.11


##### Add a 'Stock Price' column to the main_dataframe

In [125]:
# Get API url for historical stock price
url_stock_price = ("https://financialmodelingprep.com/api/v3/historical-price-full/?serietype=line") # 63

# Add ticker symbol into data URL - this extracts data from the website reference
api_call = name_API_stock_price(ticker, url_stock_price)

# Take the website data from json into a pandas dataframe
stock_price = get_jsonparsed_data(api_call)
stock_data_stock_price = stock_price["historical"]
#stock_data_stock_price

# Reverse list to match descending order of main_dataframe
stock_data_stock_price.reverse()

main_dataframe

,date,Revenue Growth,EPS,Stock Price
0,2019-06-29,0.0102,2.2,197.64
1,2019-03-30,-0.0511,2.47,154.966
2,2018-12-29,-0.0451,4.22,223.135
3,2018-09-29,0.1963,2.94,139.844
4,2018-06-30,0.173,2.36,138.458
5,2018-03-31,0.1558,2.75,111.392
6,2017-12-30,0.1269,3.92,107.34
7,2017-09-30,0.1222,2.09,89.8407
8,2017-07-01,0.072,1.68,101.859
9,2017-04-01,0.0463,2.11,102.307


### Part 3 - Data Visualization

In [126]:
# PLOTTING...
"""
x = (clean_dataframe_income_statement['date'])[:5]
y = (clean_dataframe_income_statement['EPS'])[:5]
# Graph date vs EPS plot
plt.scatter(x, y)
plt.ylabel('Earnings Per Share')
plt.xlabel('Date')
plt.show()
"""

"\nx = (clean_dataframe_income_statement['date'])[:5]\ny = (clean_dataframe_income_statement['EPS'])[:5]\n# Graph date vs EPS plot\nplt.scatter(x, y)\nplt.ylabel('Earnings Per Share')\nplt.xlabel('Date')\nplt.show()\n"